<a href="https://colab.research.google.com/github/cweiqiang/wq.github.io/blob/main/Cheatsheet_PySpark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SparkSQL
Spark SQL is Apache Spark's module

for working with structured data.

# Section 1: Initializing SparkSession

A SparkSession can be used create DataFrame, register DataFrame as tables,

execute SQL over tables, cache tables, and read parquet files.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Section 2: Creating DataFrames

## From RDDs

In [ ]:
from pyspark.sql.types import *

### Infer Schema

In [ ]:
sc = spark.sparkContext
lines = sc.textFile("people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0],age=int(p[1])))
peopledf = spark.createDataFrame(people)

### Specify Schema

In [ ]:
people = parts.map(lambda p: Row(name=p[0],
                                 age=int(p[1].strip())))

schemaString = "name age"

fields = [StructField(field_name, StringType(), True) for
          field_name in schemaString.split()]

schema = StructType(fields)

spark.createDataFrame(people, schema).show()

## From Spark Data Sources

### JSON

In [ ]:
df = spark.read.json("customer.json")

df.show()
df2 = spark.read.load("people.json", format="json")

## Parquet files

In [ ]:
df3 = spark.read.load("users.parquet")

### TXT files

In [ ]:
df4 = spark.read.text("people.txt")

# Section 3: Filter

In [ ]:
#Filter entries of age, only keep those records of which the values are >24
df.filter(df["age"]>24).show()

# Section 4: Duplicate Values

In [ ]:
df = df.dropDuplicates()

# Section 5: Queries

In [ ]:
from pyspark.sql import functions as F

### Select

In [ ]:
df.select("firstName").show() #Show all entries in firstName column

df.select("firstName","lastName") \
  .show()

df.select("firstName", #Show all entries in firstName, age and type
          "age", 
          explode("phoneNumber") \
          .alias("contactInfo")) \
  .select("contactInfo.type", 
          "firstName", 
          "age") \
  .show()

df.select(df["firstName"],df["age"]+ 1)  #Show all entries in firstName and age, 
  .show()                                # add 1 to the entries of age

df.select(df['age'] > 24).show() #Show all entries where age >24

### When

In [ ]:
df.select("firstName", #Show firstName and 0 or 1 depending on age >30
          F.when(df.age > 30, 1) \ 
          .otherwise(0)) \
  .show()

df[df.firstName.isin("Jane","Boris")] #Show firstName if in the given options
              .collect()

### Like

In [ ]:
df.select("firstName", #Show firstName, and lastName is TRUE if lastName is like Smith
          df.lastName.like("Smith")) \ 
  .show()

### Startswith - Endswith

In [ ]:
df.select("firstName", #Show firstName, and TRUE if lastName starts with Sm
          df.lastName \ 
            .startswith("Sm")) \
  .show()

df.select(df.lastName.endswith("th"))\ #Show last names ending in th\
  .show()

### Sub string

In [ ]:
df.select(df.firstName.substr(1, 3) \ #Return substrings of firstName
          .alias("name")) \
  .collect()

### Between

In [ ]:
df.select(df.age.between(22, 24)) \ #Show age: values are TRUE if between 22 and 24
  .show()

# Section 5: Add, Update & Remove Columns

### Adding Columns

In [ ]:
df = df.withColumn('city',df.address.city) \
       .withColumn('postalCode',df.address.postalCode) \
       .withColumn('state',df.address.state) \
       .withColumn('streetAddress',df.address.streetAddress) \
       .withColumn('telePhoneNumber', explode(df.phoneNumber.number)) \
       .withColumn('telePhoneType', explode(df.phoneNumber.type))

## Updating Columns

In [ ]:
df = df.withColumnRenamed('telePhoneNumber', 'phoneNumber')

## Removing Columns

In [ ]:
df = df.drop("address", "phoneNumber")
df = df.drop(df.address).drop(df.phoneNumber)

# Section 6: Missing & Replacing Values

In [ ]:
df.na.fill(50).show() #Replace null values

df.na.drop().show() #Return new df omitting rows with null values

df.na \ #Return new df replacing one value with another
  .replace(10, 20) \ 
  .show()

# Section 7: GroupBy

In [ ]:
df.groupBy("age")\ #Group by age, count the members in the groups
  .count() \ 
  .show()

# Section 8: Sort

In [ ]:
peopledf.sort(peopledf.age.desc()).collect()

df.sort("age", ascending=False).collect()

df.orderBy(["age","city"],ascending=[0,1])\
  .collect()

# Section 9: Repartitioning

In [ ]:
df.repartition(10)\ #df with 10 partitions
  .rdd \
  .getNumPartitions()

df.coalesce(1).rdd.getNumPartitions() #df with 1 partition

# Section 10: Running Queries Programmatically

### Registering DataFrames as Views


In [ ]:
peopledf.createGlobalTempView("people")

df.createTempView("customer")

df.createOrReplaceTempView("customer")

### Query Views

In [ ]:
df5 = spark.sql("SELECT * FROM customer").show()

peopledf2 = spark.sql("SELECT * FROM global_temp.people")\
                 .show()

# Section 11: Inspect Data

In [ ]:
df.dtypes #Return df column names and data types

df.show() #Display the content of df

df.head() #Return first n rows

df.first() #Return first row

df.take(2) #Return the first n rows >>> df.schema Return the schema of df

df.describe().show() #Compute summary statistics >>> df.columns Return the columns of df

df.count() #Count the number of rows in df

df.distinct().count() #Count the number of distinct rows in df

df.printSchema() #Print the schema of df

df.explain() #Print the (logical and physical) plans

# Section 12: Output

### Data Structures

In [ ]:
rdd1 = df.rdd #Convert df into an RDD

df.toJSON().first() #Convert df into a RDD of string

df.toPandas() #Return the contents of df as Pandas DataFrame

### Write & Save to Files


In [ ]:
df.select("firstName", "city")\
  .write \
  .save("nameAndCity.parquet")

df.select("firstName", "age") \
  .write \
  .save("namesAndAges.json",format="json")

# Section 13: Stopping SparkSession

In [ ]:
spark.stop()